In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = './cifar-10-batches-py'
print("1. Structure of CIFAR_DIR {0}".format(os.listdir(CIFAR_DIR)))

1. Structure of CIFAR_DIR ['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [6]:
def loadData(filename):
    '''
    Read data from filename
    '''
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    '''
    CifarData is a wrapper that process CIFAR-10 Data
    '''
    def __init__(self, filenames, need_shuffle):
        '''
        The constructor takes in a list of files and read all data and labels into self._data member
        @param filenames - list of files that contains all data
        @param need_shuffle - a boolean flag indicating whether order of data needs to be shuffled. 
                              `True` of training set and `False` for testing set
        '''
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = loadData(filename)
            for d, l in zip(data, labels):
                if l in [0, 1]:
                    all_data.append(d)
                    all_labels.append(l)
        
        self._data = np.vstack(all_data) / 127.5 - 1
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
            # When we process the data by Mini Batch, this is a cursor indicating where
            # we are in the dataset
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        '''
        Get a random permutation of all indices between 0 and _num_examples, and apply 
        this random permutation of indices to `self._data` and `self._labels`.
        '''
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        '''
        Generate a batch of data for processing. For training set, if end index is larger 
        than `self._num_examples`, reshuffle the data and start over from index 0. For 
        testing set, if end index is out of bound, raise an exception.
        @param batch_size - Number of samples for the batch of data and labels to return
        '''
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("Do not have enough testing samples for next batch")
                
        if end_indicator > self._num_examples:
            raise Exception("Batch size is larger than _num_examples")
            
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    def reset(self):
        '''
        `_shuffle_data` and reset `_indicator` to 0
        '''
        self._shuffle_data()
        self._indicator = 0
    
    def getDataShape(self):
        return self._data.shape
    
    def getLabelShape(self):
        return self._labels.shape
    
print("2. Basic test for `loadData` and `CifarData` class.")
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range (1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
print("Train data shape = {0}, labels shape = {1}".
      format(train_data.getDataShape(), train_data.getLabelShape()))

test_data = CifarData(test_filenames, False)
print("Test data shape = {0}, labels shape = {1}".
      format(test_data.getDataShape(), test_data.getLabelShape()))

batch_data, batch_labels = train_data.next_batch(10)
print("Batch Data = {0}, Batch Labels = {1}".format(batch_data, batch_labels))

2. Basic test for `loadData` and `CifarData` class.
Train data shape = (10000, 3072), labels shape = (10000,)
Test data shape = (2000, 3072), labels shape = (2000,)
Batch Data = [[-0.81960784 -0.81176471 -0.79607843 ... -0.38039216 -0.43529412
  -0.49803922]
 [ 0.2         0.18431373  0.19215686 ... -0.09019608 -0.09019608
  -0.14509804]
 [-0.05098039 -0.05882353 -0.05882353 ... -0.01176471 -0.01176471
  -0.01176471]
 ...
 [ 0.21568627  0.18431373  0.19215686 ... -0.21568627 -0.23137255
  -0.24705882]
 [ 0.02745098  0.00392157 -0.01176471 ... -0.23921569 -0.24705882
  -0.23921569]
 [-1.         -0.99215686 -0.99215686 ... -0.1372549  -0.1372549
  -0.12941176]], Batch Labels = [0 0 0 0 0 0 0 0 0 1]


In [3]:
# Define x and y
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

# Define weights w and intercept b
# w shape is (3072, 1) and it is initialized as a randome value distributed normally
# b shape is (1,) and all of them are initialized as 0.0
w = tf.get_variable('w', [x.get_shape()[-1], 1], initializer = tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [1], initializer = tf.constant_initializer(0.0))

# y_pred, predict and y_actual are both [None, 1] shape
y_prob = tf.nn.sigmoid(tf.matmul(x, w) + b)
y_predict = tf.cast(y_prob > 0.5, tf.int64)
y_actual = tf.reshape(y, (-1, 1))
y_actual_float = tf.cast(y_actual, tf.float32)

# Calculate loss and accuracy
loss = tf.reduce_mean(tf.square(y_actual_float - y_prob))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_predict, y_actual), tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
print("3. Apply TensorFlow model to CIFAR-10 Data.")
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
train_data.reset()

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op],
                                        feed_dict = {x: batch_data, y: batch_labels})
        # Every 500 steps print loss and accuracy
        if (i + 1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, accuracy: %4.5f' \
                 % (i + 1, loss_val, acc_val))
        
        # Every 5,000 steps run model against test model for accuracy
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_accuracies = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], 
                                        feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_accuracies.append(test_acc_val)
            test_accuracy = np.mean(all_test_accuracies)
            print('[Test ] step: %d, accuracy: %4.5f' % (i + 1, test_accuracy))

3. Apply TensorFlow model to CIFAR-10 Data.
[Train] step: 500, loss: 0.15088, accuracy: 0.85000
[Train] step: 1000, loss: 0.31448, accuracy: 0.65000
[Train] step: 1500, loss: 0.26977, accuracy: 0.70000
[Train] step: 2000, loss: 0.20061, accuracy: 0.80000
[Train] step: 2500, loss: 0.18322, accuracy: 0.80000
[Train] step: 3000, loss: 0.24877, accuracy: 0.75000
[Train] step: 3500, loss: 0.19999, accuracy: 0.80000
[Train] step: 4000, loss: 0.28435, accuracy: 0.70000
[Train] step: 4500, loss: 0.05069, accuracy: 0.95000
[Train] step: 5000, loss: 0.00188, accuracy: 1.00000
[Test ] step: 5000, accuracy: 0.79950
[Train] step: 5500, loss: 0.00000, accuracy: 1.00000
[Train] step: 6000, loss: 0.00886, accuracy: 1.00000
[Train] step: 6500, loss: 0.15020, accuracy: 0.85000
[Train] step: 7000, loss: 0.29994, accuracy: 0.70000
[Train] step: 7500, loss: 0.04852, accuracy: 0.95000
[Train] step: 8000, loss: 0.29866, accuracy: 0.70000
[Train] step: 8500, loss: 0.05041, accuracy: 0.95000
[Train] step: 9000

[Train] step: 71500, loss: 0.20000, accuracy: 0.80000
[Train] step: 72000, loss: 0.17874, accuracy: 0.80000
[Train] step: 72500, loss: 0.20312, accuracy: 0.80000
[Train] step: 73000, loss: 0.20010, accuracy: 0.80000
[Train] step: 73500, loss: 0.06265, accuracy: 0.95000
[Train] step: 74000, loss: 0.10049, accuracy: 0.90000
[Train] step: 74500, loss: 0.10022, accuracy: 0.90000
[Train] step: 75000, loss: 0.10000, accuracy: 0.90000
[Test ] step: 75000, accuracy: 0.82050
[Train] step: 75500, loss: 0.39839, accuracy: 0.60000
[Train] step: 76000, loss: 0.45026, accuracy: 0.55000
[Train] step: 76500, loss: 0.10165, accuracy: 0.90000
[Train] step: 77000, loss: 0.19551, accuracy: 0.80000
[Train] step: 77500, loss: 0.15127, accuracy: 0.85000
[Train] step: 78000, loss: 0.15635, accuracy: 0.85000
[Train] step: 78500, loss: 0.30005, accuracy: 0.70000
[Train] step: 79000, loss: 0.15003, accuracy: 0.85000
[Train] step: 79500, loss: 0.00006, accuracy: 1.00000
[Train] step: 80000, loss: 0.15202, accurac